In [3]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import os

# Cargar archivo
file_path = "Actualización Censo Resguardo Indigena de Guachucal - PARA USO INTERNO.xlsx"

# Verificar si el archivo existe
if not os.path.exists(file_path):
    print(f"Error: El archivo '{file_path}' no se encontró.")
    # Salir del script o manejar el error adecuadamente
    exit() # O raise FileNotFoundError(f"File not found: {file_path}")


try:
    df = pd.read_excel(file_path, sheet_name=0, skiprows=6)
except Exception as e:
    print(f"Error al leer el archivo Excel: {e}")
    exit()

# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# Eliminar filas vacías en columnas clave (usar subset para evitar errores si la columna no existe temporalmente)
required_columns = ['FAMILIA', 'INTEGRANTES']
# Verificar si las columnas existen antes de usarlas en subset
if not all(col in df.columns for col in required_columns):
     print(f"Error: Faltan columnas esenciales ('FAMILIA' o 'INTEGRANTES') en el archivo.")
     exit()

df = df.dropna(subset=required_columns)


# Convertir a enteros (manejar posibles errores de conversión si hay texto)
try:
    df['FAMILIA'] = df['FAMILIA'].astype(int)
    df['INTEGRANTES'] = df['INTEGRANTES'].astype(int)
except ValueError as e:
    print(f"Advertencia: Error al convertir columnas 'FAMILIA' o 'INTEGRANTES' a entero. Esto puede deberse a valores no numéricos. Esos valores podrían causar problemas.")
    # Una alternativa más robusta es usar:
    # df['FAMILIA'] = pd.to_numeric(df['FAMILIA'], errors='coerce').fillna(0).astype(int) # O manejar los NaNs de otra forma
    # df['INTEGRANTES'] = pd.to_numeric(df['INTEGRANTES'], errors='coerce').fillna(0).astype(int)
    # Por ahora, continuamos con la conversión original y mostramos advertencia.

# Reasignar códigos de familia
# Genera el 'Nuevo_Codigo_Familia' agrupando filas consecutivas con el mismo código original
nuevo_codigo = []
codigo_actual = 0
ultimo_codigo = None

# Asegurarse de que el DataFrame no esté vacío después de dropna
if not df.empty:
    for _, row in df.iterrows():
        # Comienza un nuevo código cuando el valor de FAMILIA cambia respecto al anterior
        if row['FAMILIA'] != ultimo_codigo:
            codigo_actual += 1
            ultimo_codigo = row['FAMILIA']
        nuevo_codigo.append(codigo_actual)

    df['Nuevo_Codigo_Familia'] = nuevo_codigo
else:
    print("El DataFrame está vacío después de eliminar filas vacías en columnas clave.")
    # Decidir qué hacer si el DF está vacío, quizás salir.
    exit()


# VALIDACIÓN 1: Verificar que el número de integrantes sea correcto
# Cuenta cuántas filas hay por cada 'Nuevo_Codigo_Familia' y compara con 'INTEGRANTES'
df['Validacion_Integrantes'] = df.groupby('Nuevo_Codigo_Familia')['Nuevo_Codigo_Familia'].transform('count')
df['Error_Integrantes'] = df['Validacion_Integrantes'] != df['INTEGRANTES']
df['Error_Integrantes'] = df['Error_Integrantes'].replace({True: 'ERROR - Revisar Integrantes', False: ''})


# --- NUEVA FUNCIONALIDAD: VALIDACIÓN 2: Verificar el orden de Parentezco ---

# Verificar si la columna 'Parentezco' existe antes de procesarla
if 'PARENTEZCO' not in df.columns:
    print("Error: La columna 'PARENTEZCO' no se encontró en el archivo.")
    # Crear una columna de error vacía para que el resto del script funcione
    df['Error_Orden_Parentezco'] = ''
    print("La validación de orden de parentesco no se realizará.")
    # Saltamos la lógica de validación de orden
    parentezco_validation_skipped = True
else:
    parentezco_validation_skipped = False
    # Renombrar la columna a 'Parentezco' si está en mayúsculas para consistencia interna
    df.rename(columns={'PARENTEZCO': 'Parentezco'}, inplace=True)

    # 1. Definir el orden esperado de parentesco
    # Asignamos un rango numérico a cada tipo de parentesco esperado.
    # Un número menor significa una posición más alta en la jerarquía esperada.
    parentezco_order_map = {
        'CF': 0,    # Cabeza de Familia
        'ES': 1,    # Esposo/Esposa
        'HI': 2,    # Hijo/Hija
        'NI': 3,    # Nieto/Nieta
        'SO': 4     # Sobrino/Sobrina
        # Puedes añadir otros si son válidos y definir su posición jerárquica.
        # Si un valor no está en este mapa, se considerará "desconocido" o fuera de la jerarquía principal.
    }

    # Valor de rango alto para parentescos no definidos en el mapa o celdas vacías (NaN)
    # Esto ayuda a que la lógica de chequeo de "rank descendente" funcione,
    # ya que un parentesco conocido (rank bajo) después de uno desconocido (rank 99)
    # no activaría el error de "rank descendente". Sin embargo, también añadiremos
    # la validación específica del primer elemento.
    UNKNOWN_RANK = 99


    # Función para verificar el orden dentro de cada grupo familiar
    def check_parentezco_order(group_df, rank_map, unknown_rank):
        # Obtener la columna de Parentezco para el grupo, manejando NaNs
        # Convertimos a lista para facilitar la iteración secuencial
        parentezco_list = group_df['Parentezco'].fillna('').tolist() # Reemplazar NaN con cadena vacía

        # Si el grupo está vacío o tiene solo una persona, no hay error de orden por secuencia
        if not parentezco_list or len(parentezco_list) <= 1:
            # Aún debemos revisar si la única persona es CF si aplica
            pass # La validación del primer elemento se hará abajo

        error_found = False
        current_known_rank = -1 # Rango del último parentesco conocido visto en la secuencia

        # Validar la secuencia de rangos: si un parentesco conocido aparece con un rango menor
        # que el parentesco conocido anterior en la secuencia, hay un error.
        for i, code in enumerate(parentezco_list):
            # Obtener el rango del código actual. Si no está en el mapa, usar UNKNOWN_RANK.
            rank = rank_map.get(code, unknown_rank)

            if rank < unknown_rank: # Si es un parentesco conocido (no desconocido/vacío)
                # Comprobar si este parentesco conocido tiene un rango menor que el último parentesco conocido visto
                if rank < current_known_rank:
                    error_found = True
                    break # Encontramos un error, no necesitamos revisar más en este grupo
                # Actualizar el rango del último parentesco conocido visto
                current_known_rank = rank
            # Si es un parentesco desconocido (rank == unknown_rank), no actualizamos current_known_rank.
            # Esto permite, por ejemplo, [CF, DESCONOCIDO, HIJO] sea válido si HIJO tiene el rango correcto después de CF.

        # Validar si el primer elemento *con un parentesco conocido* es 'CF'.
        # Esto maneja casos donde el primer elemento es, por ejemplo, 'HI' o 'ES'.
        first_known_code_in_group = None
        for code in parentezco_list:
            if code in rank_map:
                first_known_code_in_group = code
                break # Encontramos la primera persona con un parentesco conocido

        # Si existe una persona con un parentesco conocido en el grupo
        if first_known_code_in_group is not None:
             # Si la primera persona conocida no es 'CF', y 'CF' está en nuestro mapa (es esperado como inicio)
             if first_known_code_in_group != 'CF' and 'CF' in rank_map:
                  error_found = True


        return error_found

    # Aplicar la función de validación de orden a cada grupo familiar
    # Esto retornará una Serie con True/False por cada Nuevo_Codigo_Familia
    # Usamos .apply() en el objeto agrupado
    group_order_errors = df.groupby('Nuevo_Codigo_Familia').apply(
        lambda x: check_parentezco_order(x, parentezco_order_map, UNKNOWN_RANK)
    )

    # Mapear los resultados de los grupos de vuelta a cada fila del DataFrame original
    # Creamos una nueva columna temporal 'Order_Error_Flag' en el DataFrame
    # df['Nuevo_Codigo_Familia'] es una Serie que podemos mapear
    df['Order_Error_Flag'] = df['Nuevo_Codigo_Familia'].map(group_order_errors)

    # Crear la columna de error final para el formato
    df['Error_Orden_Parentezco'] = df['Order_Error_Flag'].apply(lambda x: 'ERROR - Orden Parentezco' if x else '')

# --- Fin de la NUEVA FUNCIONALIDAD ---


# Definir nombre del archivo de salida
# os.path.splitext() separa nombre y extensión
nombre_base, extension = os.path.splitext(file_path)
output_file = f"VALIDADO_{os.path.basename(nombre_base)}.xlsx" # Usar solo el nombre del archivo original


# Guardar archivo con formato en Excel usando openpyxl
# Esto nos permite trabajar directamente con el objeto workbook para aplicar formato condicional
try:
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        # Guardar el DataFrame completo (incluyendo las nuevas columnas de error)
        # Asegurarse de que las columnas de error que se usarán para formatear estén presentes
        cols_to_save = list(df.columns)
        if 'Error_Integrantes' not in cols_to_save: cols_to_save.append('Error_Integrantes')
        if 'Error_Orden_Parentezco' not in cols_to_save: cols_to_save.append('Error_Orden_Parentezco')

        df.to_excel(writer, sheet_name="Censo Ordenado", index=False, columns=cols_to_save)

        workbook = writer.book
        worksheet = writer.sheets["Censo Ordenado"]

    # Si necesitamos cargar de nuevo para formato (a veces es necesario si el with block cierra el archivo)
    # Descomenta las siguientes 2 líneas si el formato no se aplica
    # workbook = load_workbook(output_file)
    # worksheet = workbook["Censo Ordenado"]

    # Definir colores de relleno
    # Color para error de número de integrantes (Rojo claro)
    fill_integrantes_error = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid") # Un rojo un poco más claro
    # Color para error de orden de parentesco (Amarillo)
    fill_orden_error = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

    # Aplicar color a toda la fila donde haya error
    # Iteramos por cada fila en el DataFrame para obtener la información de error
    # start=2 porque en Excel la primera fila es 1 (encabezado), la segunda es 2 (primer dato)
    for r_idx, row_data in df.iterrows():
        excel_row_idx = r_idx + 2 # Calcular el índice de fila en Excel (1-based, más el encabezado)

        # Obtener el valor de las columnas de error para la fila actual
        # Usamos .get() con un valor por defecto para evitar errores si la columna no se creó (ej: si Parentezco faltaba)
        error_orden = row_data.get('Error_Orden_Parentezco', '')
        error_integrantes = row_data.get('Error_Integrantes', '')

        # Prioridad: Si hay error de orden, aplicar AMARILLO
        if error_orden == "ERROR - Orden Parentezco":
            fill_to_apply = fill_orden_error
        # Si NO hay error de orden, pero sí hay error de integrantes, aplicar ROJO
        elif error_integrantes == "ERROR - Revisar Integrantes":
            fill_to_apply = fill_integrantes_error
        else:
            fill_to_apply = None # No aplicar relleno si no hay error

        # Si se determinó un relleno, aplicarlo a todas las celdas de la fila en la hoja de cálculo
        if fill_to_apply:
            # len(df.columns) es el número de columnas del DataFrame, +1 para el índice 1-based de openpyxl
            for col_idx in range(1, len(df.columns) + 1):
                worksheet.cell(row=excel_row_idx, column=col_idx).fill = fill_to_apply

    # Guardar los cambios de formato en el archivo Excel
    workbook.save(output_file)

    print(f"Archivo guardado exitosamente en {output_file}.")
    print("Filas con error en número de integrantes marcadas en ROJO.")
    if not parentezco_validation_skipped:
        print("Familias con error en orden de parentesco marcadas en AMARILLO.")
    else:
         print("La validación de orden de parentesco fue omitida por falta de la columna 'PARENTEZCO'.")

except Exception as e:
    print(f"Error al guardar o formatear el archivo Excel: {e}")

Archivo guardado exitosamente en VALIDADO_Actualización Censo Resguardo Indigena de Guachucal - PARA USO INTERNO.xlsx.
Filas con error en número de integrantes marcadas en ROJO.
Familias con error en orden de parentesco marcadas en AMARILLO.
